In [1]:
import sys
sys.path.append('../')

In [2]:
%load_ext autoreload
%autoreload 2

# Imports

In [3]:
import os
import torch
from langchain.document_loaders import PyMuPDFLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [9]:
PATH_PROJECT = "C:/Users/victo/Documents/LLM/mva-llm-project/" 
CHROMA_PERSIST_DIR = os.path.join(PATH_PROJECT, "data/chroma_db/")

if not os.path.exists(CHROMA_PERSIST_DIR) or not os.listdir(CHROMA_PERSIST_DIR):
    print("🆕 Création de la base Chroma et indexation des documents...")

    # Charger les fichiers PDF
    data_dir = os.path.join(PATH_PROJECT, "data/")
    documents = []

    for file in os.listdir(data_dir):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(data_dir, file)
            pdf_loader = PyMuPDFLoader(pdf_path)
            documents.extend(pdf_loader.load())

    # Diviser les documents en chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)

    embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",
                                       model_kwargs={'device': device}, encode_kwargs={'device': device})

    # Initialiser Chroma avec les embeddings 
    vectorstore = Chroma.from_documents(docs, embedding_model, persist_directory=CHROMA_PERSIST_DIR)
    vectorstore.persist()
    print("Base Chroma sauvegardée avec succès")

else:
    print("Chargement de la base Chroma existante...")
    embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",
                                       model_kwargs={'device': device}, encode_kwargs={'device': device})
    vectorstore = Chroma(persist_directory=CHROMA_PERSIST_DIR, embedding_function=embedding_model)

print("Base Chroma prête à être utilisée")

Chargement de la base Chroma existante...
Base Chroma prête à être utilisée


In [6]:
query = "What are the symptoms of depression ?"
retrieved_docs = vectorstore.similarity_search(query, k=3)

for doc in retrieved_docs:
    print(doc.page_content)  # Afficher les documents les plus pertinents

259
Mood disorders
•	 Depressive symptoms include depressed mood or markedly diminished interest or 
pleasure in activities.
•	 There are multiple anxiety symptoms, which may include feeling nervous, anxious or on 
edge, not being able to control worrying thoughts, fear that something awful will happen, 
having trouble relaxing, muscle tension or sympathetic autonomic symptoms.
•	 Neither the depressive nor the anxiety symptoms – considered separately – are sufficiently
guilt, anger, denial, blame, difficulty accepting the death, feeling one has lost a part of one’s self, 
inability to experience positive mood, emotional numbness, difficulty in engaging with social or 
other activities). Some common symptoms of prolonged grief disorder are similar to those observed 
in a depressive episode (e.g. sadness, loss of interest in activities, social withdrawal, feelings of guilt,
•	 The symptoms include:
•	 at least one affective symptom such as mood lability, irritability, depressed mood or 

In [ ]:
import ollama

context = "\n\n".join([doc.page_content for doc in retrieved_docs])
prompt = f"Contexte:\n{context}\n\nQuestion: {query}\nRéponse:"

response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])
print(response['message'].content)

 Depression symptoms include depressed mood or markedly diminished interest or pleasure in activities, feelings of guilt or worthlessness, fatigue or loss of energy, difficulty concentrating, changes in appetite or sleep, and recurrent thoughts of death or suicide. Additionally, there may be somatic or cognitive symptoms such as lethargy, joint pain, overeating, hypersomnia, breast tenderness, swelling of extremities, concentration difficulties or forgetfulness. It's important to note that these symptoms can vary in severity and may not always be present for every individual experiencing depression. If you or someone else is struggling with these symptoms, it's essential to seek help from a healthcare professional.
